In [ ]:
%pip install sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

In [4]:
sentences = [
    "it caught him off guard that space smelled of seared steak",
    "she could not decide between painting her teeth or brushing her nails",
    "he thought there'd be sufficient time is he hid his watch",
    "the bees decided to have a mutiny against their queen",
    "the sign said there was road work ahead so she decided to speed up",
    "on a scale of one to ten, what's your favorite flavor of color?",
    "flying stinging insects rebelled in opposition to the matriarch"
]

In [ ]:
embeddings = model.encode(sentences)
embeddings.shape

In [ ]:
from sentence_transformers.util import cos_sim

scores = cos_sim(embeddings[-1], embeddings[:-1])

print(sentences[-1])
for i, score in enumerate(scores[0]):
    print(f"{round(score.item(), 4)} | {sentences[i]}")